# Download JWST NIRSpec MOS data from MAST

https://github.com/spacetelescope/dat_pyinthesky/blob/main/jdat_notebooks/NIRSpec_MAST_Query/NIRSpec_MAST_Query.ipynb

https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries

https://mast.stsci.edu/api/v0/_c_a_o_mfields.html

### ERO program 2736: SMACS0723

https://webbtelescope.org/contents/news-releases/2022/news-2022-035

Observations:
* two pointings: s007, s008  
* G235M & G395M
* 3 slitlet nods x 2 integrations x 20 groups NRSIRS2 = 8841 s exposure time = 2.5 hours


In [1]:
import astroquery
print('astroquery version', astroquery.__version__)
from astroquery.mast import Observations, Mast

from astropy.table import Table, vstack, unique
import os
import numpy as np

astroquery version 0.4.6


In [2]:
def download_jwst_files(filenames, download_dir, mast_dir='mast:jwst/product'):
    # Download data
    # https://astroquery.readthedocs.io/en/latest/mast/mast_obsquery.html#downloading-data
    #manifest = Observations.download_products(chosen_obsIDs, download_dir=DOWNLOAD_DIR)  # , flat=True

    downloaded_files = []
    os.makedirs(download_dir, exist_ok=True)
    for filename in filenames:
        mast_path  = os.path.join(mast_dir,     filename)
        local_path = os.path.join(download_dir, filename)
        Observations.download_file(mast_path,   local_path=local_path)
        downloaded_files.append(local_path)

    return downloaded_files

In [3]:
def allin(elements, list_or_string):
    if type(elements) == str:
        elements = elements.split()
    for element in elements:
        if element not in list_or_string:
            return False
    return True

def anyin(elements, list_or_string):
    if type(elements) == str:
        elements = elements.split()
    for element in elements:
        if element in list_or_string:
            return True
    return False

def select_files(all_files, search_strings=[]):
    chosen_files = [file for file in all_files if allin(search_strings, file)]
    #if len(chosen_files) == 1:
    #    chosen_files = chosen_files[0]
    return chosen_files    

https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/file_naming.html

# Start here

## MAST query files from this JWST program

https://astroquery.readthedocs.io/en/latest/api/astroquery.mast.ObservationsClass.html

In [4]:
#Observations.get_metadata('observations')  # list all criteria that may be used in query_criteria below

In [5]:
obs_table = Observations.query_criteria(obs_collection = 'JWST', 
                                        proposal_id = 2736,  # program number 2736: ERO SMACS0723
                                        instrument_name = 'NIRSPEC/MSA',
                                        filters = 'g395m',
                                        dataproduct_type = 'spectrum',
                                       )

products_list = Observations.get_product_list(obs_table)  # all output products including images, catalogs, etc.

In [6]:
products_list[:3]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str8,str40,str71,str1,str70,str9,str28,str8,str1,str7,str6,str4,str52,int64,str9,str6,int64
100975615,JWST,spectrum,jw02736-o007_s00001_nirspec_f290lp-g395m,source/target (L3) : association generator,D,mast:JWST/product/jw02736-o007_20231116t144653_spec3_00001_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.11.4,2736,jw02736-o007_20231116t144653_spec3_00001_asn.json,5501,100975615,PUBLIC,3
100975615,JWST,spectrum,jw02736-o007_s00001_nirspec_f290lp-g395m,source/target (L3) : association pool,D,mast:JWST/product/jw02736_20231116t144653_pool.csv,INFO,Minimum Recommended Products,POOL,--,CALJWST,1.11.4,2736,jw02736_20231116t144653_pool.csv,177115,100975615,PUBLIC,3
100975615,JWST,spectrum,jw02736-o007_s00001_nirspec_f290lp-g395m,Preview-Full,D,mast:JWST/product/jw02736-o007_s00001_nirspec_f290lp-g395m_cal.jpg,PREVIEW,--,--,--,CALJWST,1.11.4,2736,jw02736-o007_s00001_nirspec_f290lp-g395m_cal.jpg,1038,100975615,PUBLIC,3


In [7]:
all_filenames = products_list['obs_id']
#all_filenames

In [8]:
data_products = Observations.filter_products(products_list, 
                                            productType = 'INFO', 
                                            productSubGroupDescription = 'ASN',  # UNCAL, RATE, RATEINTS, CAL, I2D, X1D, S2D, ASN
                                            type = 'S', # 'S': all objects;  'D': individual object
                                            )  

In [9]:
data_products[:2]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str8,str40,str71,str1,str70,str9,str28,str8,str1,str7,str6,str4,str52,int64,str9,str6,int64
87600657,JWST,image,jw02736007001_02101_00001_nrs1,source/target (L3) : association generator,S,mast:JWST/product/jw02736-o007_20231116t144653_image2_00008_asn.json,INFO,--,ASN,--,CALJWST,1.5.3,2736,jw02736-o007_20231116t144653_image2_00008_asn.json,1417,87602347,PUBLIC,2
87600657,JWST,image,jw02736007001_02101_00001_nrs1,source/target (L3) : association generator,S,mast:JWST/product/jw02736-o007_20231116t144653_image2_00008_asn.json,INFO,--,ASN,--,CALJWST,1.5.3,2736,jw02736-o007_20231116t144653_image2_00008_asn.json,1417,87602348,PUBLIC,2


In [10]:
data_products = Observations.filter_products(products_list, 
                                            productType = 'SCIENCE', 
                                            productSubGroupDescription = 'S2D',  # UNCAL, RATE, RATEINTS, CAL, I2D, X1D, S2D, ASN
                                            type = 'S', # 'S': all objects;  'D': individual object
                                            )  

In [16]:
#data_products
len(data_products)

1164

In [12]:
# Unique filenames
chosen_files = unique(data_products, keys='productFilename')
chosen_filenames = chosen_files['productFilename']
chosen_filenames

jw02736007001_03103_00001_nrs1_s2d.fits
jw02736007001_03103_00001_nrs2_s2d.fits
jw02736007001_03103_00002_nrs1_s2d.fits
jw02736007001_03103_00002_nrs2_s2d.fits
jw02736007001_03103_00003_nrs1_s2d.fits
jw02736007001_03103_00003_nrs2_s2d.fits
jw02736008001_03103_00001_nrs1_s2d.fits
jw02736008001_03103_00001_nrs2_s2d.fits
jw02736008001_03103_00002_nrs1_s2d.fits
jw02736008001_03103_00002_nrs2_s2d.fits
jw02736008001_03103_00003_nrs1_s2d.fits


In [13]:
download_dir = '../data/s2d'

In [14]:
download_files = download_jwst_files(chosen_filenames, download_dir)

In [15]:
# Why doesn't this work?
#manifest = Observations.download_products(data_products, productType="SCIENCE", download_dir=download_dir, flat=True)
manifest = Observations.download_products(data_products, download_dir=download_dir, flat=True, mrp_only=False)

TypeError: 'bool' object is not iterable

## Download data: spectrum extraction for a single object

In [19]:
all_files = unique(products_list, keys='productFilename')
all_filenames = all_files['productFilename']
print(len(all_filenames))
all_filenames[:3]

1566


jw02736-o007_20231116t144653_image2_00001_asn.json
jw02736-o007_20231116t144653_image2_00002_asn.json
jw02736-o007_20231116t144653_image2_00003_asn.json


In [18]:
source_id = 6355  # z = 7.655 excellent spectrum w/ bright lines; better than 10612 z = 7.663 in press release
source_str = 's%05d' % source_id

chosen_files = select_files(all_filenames, search_strings=['o007', source_str, 'g395m']) #, 'x1d', 'fits'])
chosen_files

['jw02736-o007_s06355_nirspec_f290lp-g395m_cal.fits',
 'jw02736-o007_s06355_nirspec_f290lp-g395m_cal.jpg',
 'jw02736-o007_s06355_nirspec_f290lp-g395m_crf.fits',
 'jw02736-o007_s06355_nirspec_f290lp-g395m_crf.jpg',
 'jw02736-o007_s06355_nirspec_f290lp-g395m_s2d.fits',
 'jw02736-o007_s06355_nirspec_f290lp-g395m_s2d.jpg',
 'jw02736-o007_s06355_nirspec_f290lp-g395m_x1d.fits']

In [20]:
download_dir = '../data'  # Define and create directory to save downloads
download_dir = os.path.join(download_dir, source_str)
download_files = download_jwst_files(chosen_files, download_dir)

# NIRCam images

In [22]:
nircam_obs_table = Observations.query_criteria(obs_collection= 'JWST', 
                                            proposal_id = 2736,  # program number 2736: ERO SMACS0723
                                            instrument_name = 'NIRCAM/IMAGE',
                                            dataproduct_type = 'IMAGE',
                                            filters = 'f200w', 
                                            #productType = 'SCIENCE', 
                                            #productSubGroupDescription = 'i2d',  # UNCAL, RATE, RATEINTS, CAL, I2D, X1D, S2D
                                            #type = 'D', # 'S': all objects;  'D': individual object
                                            calib_level = 3,
                                              )

nircam_products_list = Observations.get_product_list(nircam_obs_table)  # all output products including images, catalogs, etc.

In [24]:
nircam_products_list[:2]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str8,str4,str5,str36,str64,str1,str68,str9,str28,str11,str1,str7,str6,str4,str50,int64,str8,str6,int64
87602476,JWST,image,jw02736-o001_t001_nircam_clear-f200w,target (L3) : segmentation map,D,mast:JWST/product/jw02736-o001_t001_nircam_clear-f200w_segm.fits,AUXILIARY,Minimum Recommended Products,SEGM,--,CALJWST,1.11.4,2736,jw02736-o001_t001_nircam_clear-f200w_segm.fits,195808320,87602476,PUBLIC,3
87602476,JWST,image,jw02736-o001_t001_nircam_clear-f200w,source/target (L3) : association generator,D,mast:JWST/product/jw02736-o001_20231116t144653_image3_00002_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.11.4,2736,jw02736-o001_20231116t144653_image3_00002_asn.json,20413,87602476,PUBLIC,3


In [25]:
all_files = unique(nircam_products_list, keys='productFilename')
all_filenames = all_files['productFilename']
chosen_files = select_files(all_filenames, search_strings=['f200w_i2d', 'fits'])
chosen_files

['jw02736-o001_t001_nircam_clear-f200w_i2d.fits']

In [26]:
download_dir = '../data'  # Define and create directory to save downloads

In [ ]:
download_files = download_jwst_files(chosen_files, download_dir)